## datasette: Explore and Publish


## ibis: Many many backends


### Polars - calmcode


In [2]:
import ibis

ibis.options.interactive = True

ModuleNotFoundError: No module named 'ibis'

In [ ]:
con_polars = ibis.polars.connect()
tbl_polars = con_polars.read_csv("../data/birthdays.csv")

In [ ]:
tbl_polars.head()

In [ ]:
def set_types(dataf):
    return dataf.mutate(dataf.date.to_date("%Y-%m-%d").name("date"))


def counter(dataf, *args):
    return (
        dataf.group_by(args)
        .agg(dataf.births.sum().name("sum"), dataf.births.mean().name("mean"))
        .order_by(args)
    )


counter(tbl_polars, "date")

### Duckdb - ibis framework


- create a duckdb file from csv `duckdb data/birthdays.ddb "CREATE TABLE birthdays AS SELECT * FROM read_csv('data/birthdays.csv');"`
- run the above from terminal
- make sure duckdb is already installed on the machine. can use homebrew


In [ ]:
import ibis
import pandas as pd

ibis.options.interactive = True

In [ ]:
df = pd.DataFrame(
    [["a", 1, 2], ["b", 3, 4]],
    columns=["one", "two", "three"],
    index=[5, 6],
)

df

,one,two,three
5,a,1,2
6,b,3,4


In [ ]:
# t = ibis.memtable(df)
# t

## neo4j: Great Graph Database


- code in Neo4j
- created `calmcode` database in neo4j

## pandas pipe: Clean Pandas Code


In [30]:
from dev.common_eda import log_step, start_pipeline

df = pd.read_csv("../data/bigmac.csv")

ModuleNotFoundError: No module named 'dev'

## pandas datetime: Datetimes in Pandas


In [2]:
import pandas as pd

df = pd.read_csv("../data/birthdays.csv")
df.dtypes

state     object
year       int64
month      int64
day        int64
date      object
wday      object
births     int64
dtype: object

In [7]:
df.dtypes

state     object
year       int64
month      int64
day        int64
date      object
wday      object
births     int64
dtype: object

In [8]:
df.head()

,state,year,month,day,date,wday,births
0,AK,1969,1,1,1969-01-01,Wed,14
1,AL,1969,1,1,1969-01-01,Wed,174
2,AR,1969,1,1,1969-01-01,Wed,78
3,AZ,1969,1,1,1969-01-01,Wed,84
4,CA,1969,1,1,1969-01-01,Wed,824


In [11]:
df.assign(
    date=lambda d: pd.to_datetime(d["date"], format="%Y-%m-%d"),
    day_of_week=lambda d: d["date"].dt.day_name(),
    minute=lambda d: d["date"].dt.minute,
    nanosecond=lambda d: d["date"].dt.nanosecond,
    day_of_year=lambda d: d["date"].dt.day_of_year,
    month_manual=lambda d: d["date"].dt.month,
    week=lambda d: d["date"].dt.isocalendar().week,
)

,state,year,month,day,date,wday,births,day_of_week,minute,nanosecond,day_of_year,month_manual,week
0,AK,1969,1,1,1969-01-01,Wed,14,Wednesday,0,0,1,1,1
1,AL,1969,1,1,1969-01-01,Wed,174,Wednesday,0,0,1,1,1
2,AR,1969,1,1,1969-01-01,Wed,78,Wednesday,0,0,1,1,1
3,AZ,1969,1,1,1969-01-01,Wed,84,Wednesday,0,0,1,1,1
4,CA,1969,1,1,1969-01-01,Wed,824,Wednesday,0,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
372550,VT,1988,12,31,1988-12-31,Sat,21,Saturday,0,0,366,12,52
372551,WA,1988,12,31,1988-12-31,Sat,157,Saturday,0,0,366,12,52
372552,WI,1988,12,31,1988-12-31,Sat,167,Saturday,0,0,366,12,52
372553,WV,1988,12,31,1988-12-31,Sat,45,Saturday,0,0,366,12,52


In [15]:
subset_df = df.assign(date=lambda d: pd.to_datetime(d["date"], format="%Y-%m-%d"))[
    ["state", "date", "births"]
].loc[lambda d: d["state"] == "CA"]

subset_df

,state,date,births
4,CA,1969-01-01,824
55,CA,1969-01-02,816
106,CA,1969-01-03,940
157,CA,1969-01-04,906
208,CA,1969-01-05,804
...,...,...,...
372304,CA,1988-12-27,1586
372355,CA,1988-12-28,1551
372406,CA,1988-12-29,1638
372457,CA,1988-12-30,1562


In [17]:
subset_df.set_index("date")

,state,births
date,,
1969-01-01,CA,824
1969-01-02,CA,816
1969-01-03,CA,940
1969-01-04,CA,906
1969-01-05,CA,804
...,...,...
1988-12-27,CA,1586
1988-12-28,CA,1551
1988-12-29,CA,1638


In [24]:
subset_df.set_index("date").resample("YE").sum()[["births"]].head()

,births
date,
1969-12-31,352858
1970-12-31,362682
1971-12-31,329816
1972-12-31,306538
1973-12-31,298062


In [31]:
(
    df[["state", "date", "births"]]
    .assign(date=lambda d: pd.to_datetime(d["date"], format="%Y-%m-%d"))
    .set_index("date")
    .groupby("state")
    .resample("2W")
    .sum()
)

state  births
state date                                            
AK    1969-01-05                    AKAKAKAKAK      88
      1969-01-19  AKAKAKAKAKAKAKAKAKAKAKAKAKAK     246
      1969-02-02  AKAKAKAKAKAKAKAKAKAKAKAKAKAK     246
      1969-02-16  AKAKAKAKAKAKAKAKAKAKAKAKAKAK     244
      1969-03-02  AKAKAKAKAKAKAKAKAKAKAKAKAKAK     260
...                                        ...     ...
WY    1988-11-13  WYWYWYWYWYWYWYWYWYWYWYWYWYWY     243
      1988-11-27  WYWYWYWYWYWYWYWYWYWYWYWYWYWY     223
      1988-12-11  WYWYWYWYWYWYWYWYWYWYWYWYWYWY     271
      1988-12-25  WYWYWYWYWYWYWYWYWYWYWYWYWYWY     238
      1989-01-08                  WYWYWYWYWYWY     114

[26673 rows x 2 columns]

In [33]:
subset_df

,state,date,births
4,CA,1969-01-01,824
55,CA,1969-01-02,816
106,CA,1969-01-03,940
157,CA,1969-01-04,906
208,CA,1969-01-05,804
...,...,...,...
372304,CA,1988-12-27,1586
372355,CA,1988-12-28,1551
372406,CA,1988-12-29,1638
372457,CA,1988-12-30,1562


In [37]:
(subset_df.set_index("date")[["births"]].rolling(10).mean())

,births
date,
1969-01-01,NaN
1969-01-02,NaN
1969-01-03,NaN
1969-01-04,NaN
1969-01-05,NaN
...,...
1988-12-27,1385.3
1988-12-28,1420.3
1988-12-29,1432.8


## polars: Faster DataFrames
